In [36]:
import numpy as np

def laplace_gamma_dp(weights, shape_param, x_values, y_values):
    """
    Thêm nhiễu vi sai bằng phân phối Laplace Gamma vào trọng số mô hình.
    """
    delta = np.max(np.abs(x_values - y_values))
    lambda_param = delta / np.mean(x_values) if np.mean(x_values) != 0 else 1.0
    
    print(delta)
    
    print("----")
    print(lambda_param)
    
    gamma_1 = np.random.gamma(shape_param, lambda_param, size=weights.shape)
    gamma_2 = np.random.gamma(shape_param, lambda_param, size=weights.shape)
    laplace_gamma_noise = gamma_1 - gamma_2
    encrypted_weights = weights + laplace_gamma_noise
    return encrypted_weights

def diffie_hellman_offset(encrypted_weights, client_id, active_clients, deltas):
    """
    Áp dụng lớp mã hóa thứ hai bằng phương pháp trao đổi khóa Diffie-Hellman.
    """
    offset = 0
    add = True
    
    for cid in active_clients:
        if cid == client_id:
            add = False
        
        if add:
            offset += deltas.get(cid, 0)
        else:
            offset -= deltas.get(cid, 0)
    
    offset_encrypted_weights = encrypted_weights + offset
    return offset_encrypted_weights

def homomorphic_encryption(offset_encrypted_weights):
    """
    Áp dụng lớp mã hóa thứ ba bằng phương pháp Homomorphic Encryption (HE).
    """
    operations = ['add', 'subtract', 'multiply', 'divide']
    operation = np.random.choice(operations)
    factor = np.random.randint(1, 10)
    
    if operation == 'divide' and np.any(offset_encrypted_weights == 0):
        operation = np.random.choice(['add', 'subtract', 'multiply'])
    
    he_encrypted_weights = np.copy(offset_encrypted_weights)
    
    for i in range(he_encrypted_weights.shape[0]):
        for j in range(he_encrypted_weights.shape[1]):
            value = he_encrypted_weights[i, j]
            if operation == 'add':
                he_encrypted_weights[i, j] = value + factor
            elif operation == 'subtract':
                he_encrypted_weights[i, j] = value - factor
            elif operation == 'multiply':
                he_encrypted_weights[i, j] = value * factor
            elif operation == 'divide':
                he_encrypted_weights[i, j] = value / factor if factor != 0 else value
    
    return he_encrypted_weights

# Ví dụ sử dụng:
np.random.seed(42)
weights = np.array([[0.5, -0.2], [0.3, 0.8]])
x_values = np.array([1.0, 2.0, 3.0, 4.0])
y_values = np.array([1.1, 1.9, 3.2, 3.8])
shape_param = 2.0

# Lớp mã hóa đầu tiên (DP Laplace Gamma)
encrypted_weights = laplace_gamma_dp(weights, shape_param, x_values, y_values)

# Giả lập danh sách client và giá trị delta
active_clients = ["A", "B", "C", "D"]
deltas = {"A": 1.5, "B": -0.5, "C": 2.0, "D": -1.0}
client_id = "C"

# Lớp mã hóa thứ hai (DK Offset)
offset_encrypted_weights = diffie_hellman_offset(encrypted_weights, client_id, active_clients, deltas)

# Lớp mã hóa thứ ba (HE Encryption)
final_encrypted_weights = homomorphic_encryption(offset_encrypted_weights)

print("Original Weights:")
print(weights)
print("\nEncrypted Weights with DP Noise:")
print(encrypted_weights)
print("\nFinal Encrypted Weights with DK Offset:")
print(offset_encrypted_weights)
print("\nFinal Encrypted Weights with HE:")
print(final_encrypted_weights)


0.20000000000000018
----
0.08000000000000007
Original Weights:
[[ 0.5 -0.2]
 [ 0.3  0.8]]

Encrypted Weights with DP Noise:
[[ 0.3195172  -0.30977932]
 [ 0.32009645  0.71299903]]

Final Encrypted Weights with DK Offset:
[[ 0.3195172  -0.30977932]
 [ 0.32009645  0.71299903]]

Final Encrypted Weights with HE:
[[ 0.05325287 -0.05162989]
 [ 0.05334941  0.11883317]]


In [7]:
import numpy as np
arr_3d = np.random.rand(3, 2, 2)
arr_3d_2 = np.random.rand(3, 2, 2)

print("-------------------------\n")
print ((arr_3d + arr_3d_2)/2)

print("000000000000000000000000\n")
print(arr_3d)
print(arr_3d_2)

-------------------------

[[[0.20950605 0.31341628]
  [0.66887103 0.6050804 ]]

 [[0.55660439 0.4672813 ]
  [0.62872016 0.64500833]]

 [[0.29331343 0.2102436 ]
  [0.49702157 0.86211546]]]
000000000000000000000000

[[[0.32894088 0.11998088]
  [0.63857247 0.68609431]]

 [[0.51516143 0.6633828 ]
  [0.51471097 0.93646422]]

 [[0.07076769 0.22655818]
  [0.94082057 0.82509948]]]
[[[0.09007121 0.50685169]
  [0.69916959 0.52406649]]

 [[0.59804735 0.2711798 ]
  [0.74272935 0.35355244]]

 [[0.51585917 0.19392902]
  [0.05322256 0.89913144]]]


In [8]:
import tenseal as ts
import numpy as np

def generate_context():
    """Tạo context HE với mã hóa CKKS."""
    context = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=8192,
        coeff_mod_bit_sizes=[60, 40, 40, 60]
    )
    context.global_scale = 2 ** 40
    context.generate_galois_keys()
    return context

def encrypt_data(context, data):
    """Mã hóa dữ liệu đầu vào."""
    return ts.ckks_vector(context, data)

def perform_operation(enc_data, operation, bl):
    """Thực hiện các phép toán trên dữ liệu mã hóa."""
    if operation == 'add':
        return enc_data + bl
    elif operation == 'sub':
        return enc_data - bl
    elif operation == 'mul':
        return enc_data * bl
    elif operation == 'div':
        return enc_data / bl
    else:
        raise ValueError("Invalid operation")

def homomorphic_encryption_pipeline(data):
    """Quy trình mã hóa đồng hình và thực hiện phép toán."""
    context = generate_context()
    encrypted_data = encrypt_data(context, data)
    
    # Chọn ngẫu nhiên một phép toán
    operation = np.random.choice(['add', 'sub', 'mul', 'div'])
    bl = np.random.randint(1, 10)
    
    if operation == 'div':
        bl = max(1, np.random.choice(data))  # Đảm bảo bl là ước số hợp lệ
    
    encrypted_result = perform_operation(encrypted_data, operation, bl)
    
    print(encrypted_result)
    return context, encrypted_result, operation, bl

def decrypt_data(context, encrypted_data):
    """Giải mã dữ liệu mã hóa."""
    return encrypted_data.decrypt()

# Dữ liệu đầu vào
data = [3.0, 7.5, 12.2, 5.6]
context, encrypted_result, operation, bl = homomorphic_encryption_pipeline(data)

dec_result = decrypt_data(context, encrypted_result)
print(f"Operation: {operation}, Bl: {bl}, Decrypted Result: {dec_result}")

Operation: add, Bl: 4, Decrypted Result: [7.000000001102751, 11.499999999953095, 16.200000000242277, 9.599999998900401]
